In [1]:
import os
import shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import cv2 as cv
from numpy.random import seed
seed(42)
import pickle

from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from glob import glob 
%matplotlib inline

Some learning parameters

In [2]:
IMG_CHANNELS = 3
#TRAIN_SIZE=80000
TRAIN_SIZE = 80000
BATCH_SIZE = 64
EPOCHS = 10


Data exploration

In [3]:
#I used cropped images
train_labels = pd.read_csv('train_labels.csv')
train_labels['label'].value_counts()


0    130908
1     89117
Name: label, dtype: int64

In [4]:
(train_labels.label.value_counts() / len(train_labels)).to_frame()

,label
0,0.594969
1,0.405031


Balancing the data

In [5]:
train_neg = train_labels[train_labels['label']==0].sample(TRAIN_SIZE,random_state=45)
train_pos = train_labels[train_labels['label']==1].sample(TRAIN_SIZE,random_state=45)

train_data = pd.concat([train_neg, train_pos], axis=0).reset_index(drop=True)

train_data = shuffle(train_data)


In [6]:
train_data['label'].value_counts()

1    80000
0    80000
Name: label, dtype: int64

In [7]:
def append_ext(fn):
    return fn+".tif"

In [8]:
y = train_data['label']
train_df, valid_df = train_test_split(train_data, test_size=0.2, random_state=45, stratify=y)

print(train_df.shape)
print(valid_df.shape)

(128000, 2)
(32000, 2)


In [9]:
train_df['id'] = train_df['id'].apply(append_ext)
valid_df['id'] = valid_df['id'].apply(append_ext)
train_df.head()

,id,label
46754,c6a2cdf3f07006fe0fbdbfb6190266e9d1925713.tif,0
57220,6618673bfdab59be5b0dcba933d3f69fa9c6edf4.tif,0
45963,9ab64d68ba53d76109a69dd8411af804cec8aa2a.tif,0
136712,b2c030b7fd8bdf0de595b352f5048ea8385fcdb2.tif,1
109273,36aa27a41fae246de8e18b695ebe7876f2647a39.tif,1


Image generator that load images in batches

In [10]:
train_datagen = ImageDataGenerator(rescale=1/255)
valid_datagen = ImageDataGenerator(rescale=1/255)

In [11]:
BATCH_SIZE = 64
train_path = './train'
train_df['label'] = train_df['label'].astype(str)
valid_df['label'] = valid_df['label'].astype(str)

train_loader = train_datagen.flow_from_dataframe(
    dataframe = train_df,
    directory = train_path,
    x_col = 'id',
    y_col = 'label',
    batch_size = BATCH_SIZE,
    seed = 42,
    shuffle = True,
    class_mode = 'categorical',
    target_size = (96,96)
)

valid_loader = valid_datagen.flow_from_dataframe(
    dataframe = valid_df,
    directory = train_path,
    x_col = 'id',
    y_col = 'label',
    batch_size = BATCH_SIZE,
    seed = 42,
    shuffle = True,
    class_mode = 'categorical',
    target_size = (96,96)
)


Found 128000 validated image filenames belonging to 2 classes.
Found 32000 validated image filenames belonging to 2 classes.


In [12]:
TR_STEPS = len(train_loader)
VA_STEPS = len(valid_loader)

print(TR_STEPS)
print(VA_STEPS)

2000
500


In [16]:
cnn = Sequential([
    Conv2D(32, (3,3), activation = 'relu', padding = 'same', input_shape=(96,96,3)),
    Conv2D(32, (3,3), activation = 'relu', padding = 'same'),
    Conv2D(32, (3,3), activation = 'relu', padding = 'same'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
    Dropout(0.2),
    BatchNormalization(),

    Conv2D(64, (3,3), activation = 'relu', padding = 'same'),
    Conv2D(64, (3,3), activation = 'relu', padding = 'same'),
    Conv2D(64, (3,3), activation = 'relu', padding = 'same'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
    Dropout(0.3),
    BatchNormalization(),
    
    Conv2D(128, (3,3), activation = 'relu', padding = 'same'),
    Conv2D(128, (3,3), activation = 'relu', padding = 'same'),
    Conv2D(128, (3,3), activation = 'relu', padding = 'same'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
    Dropout(0.4),
    BatchNormalization(),
    
    Conv2D(128, (3,3), activation = 'relu', padding = 'same'),
    Conv2D(128, (3,3), activation = 'relu', padding = 'same'),
    Conv2D(128, (3,3), activation = 'relu', padding = 'same'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
    Dropout(0.5),
    BatchNormalization(),

    Flatten(),
     Dense(128, activation='relu'),
    Dense(16, activation='relu'),
    Dense(8, activation='relu'),
    Dense(2, activation='sigmoid')
])


Round one training

In [17]:
opt = tf.keras.optimizers.Adam(0.001)
cnn.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy', tf.keras.metrics.AUC()])

In [18]:
h1 = cnn.fit(
    x = train_loader, 
    steps_per_epoch = TR_STEPS, 
    epochs = 10,
    validation_data = valid_loader, 
    validation_steps = VA_STEPS, 
    verbose = 1
)

tf.keras.backend.set_value(cnn.optimizer.learning_rate, 0.0001)

h2 = cnn.fit(
    x = train_loader, 
    steps_per_epoch = TR_STEPS, 
    epochs = 10,
    validation_data = valid_loader, 
    validation_steps = VA_STEPS, 
    verbose = 1
)

tf.keras.backend.set_value(cnn.optimizer.learning_rate, 0.00001)


h3 = cnn.fit(
    x = train_loader, 
    steps_per_epoch = TR_STEPS, 
    epochs = 10,
    validation_data = valid_loader, 
    validation_steps = VA_STEPS, 
    verbose = 1
)

Epoch 1/10
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unsupported operand type(s) for -: 'NoneType' and 'int'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unsupported operand type(s) for -: 'NoneType' and 'int'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
2000/2000 [==============================] - ETA: 0s - loss: 0.4220 - accuracy: 0.8128 - auc_1: 0.8865WARNING:tensorflow:AutoGraph could not transform <function Model.make_test_function.<locals>.test_function at 0x2e7326550> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export 

KeyboardInterrupt: 